# MFA Fatigue (Prompt Bombing)

**Repeated MFA prompts (fails/denies) followed by a success, suggesting push fatigue or social engineering.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **prompt count**, **source IP**, **new device**, **legacy auth**, **mailbox rule changes**.


## Investigation Queries

Confirm prompt storm then success. Pivot to risky sign-ins and mailbox rules.


In [ ]:
// KQL
let window=15m;
let fails=SigninLogs | where TimeGenerated>ago(window) | where UserPrincipalName=='<USER>' | where ResultType!=0 | summarize Failures=count() by IPAddress;
let succ=SigninLogs | where TimeGenerated>ago(window) | where UserPrincipalName=='<USER>' | where ResultType==0 | project SuccessTime=TimeGenerated, IPAddress;
fails | where Failures>=5 | join kind=inner (succ) on IPAddress


In [ ]:
# Splunk SPL
index=auth user=<USER> action=mfa_challenge outcome IN (fail,denied) earliest=-15m
| stats count as fails by src_ip
| where fails>=5
| join src_ip [ search index=auth user=<USER> action=login outcome=success earliest=-15m ]


In [ ]:
# Elastic (KQL/EQL)
event.category:authentication AND user.name:"<USER>" AND event.action:mfa_challenge


## Containment & Response

Reset MFA methods, revoke sessions, and block source IPs via firewall/CA policy if confirmed malicious.


In [ ]:
# PowerShell
## Reset MFA methods (MSOnline; requires module)
Reset-MsolStrongAuthenticationMethodByUpn -UserPrincipalName <USER>

## Revoke tokens (AzureAD)
Revoke-AzureADUserAllRefreshToken -ObjectId <USER_OBJECT_ID>


In [ ]:
# PowerShell
## Force password reset
Set-AzureADUserPassword -ObjectId <USER_OBJECT_ID> -PasswordProfile (New-Object -TypeName Microsoft.Open.AzureAD.Model.PasswordProfile -Property @{Password='<TEMP_PASSWORD>'; ForceChangePasswordNextLogin=$true})
